In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

In [12]:
# set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
# Function to load and prepare your data

def load_data():

  data = pd.read_csv(f'feature_engineered_dataset.csv')

  X = data.drop(columns = ['remainder__Decided to Pursue MBA?'])
  y = data['remainder__Decided to Pursue MBA?']

  print("Please replace the load_data function with your actual data loading code")
  return None, None

In [20]:
# Function to create a model with given hyperparameters

def create_model(params):

    model = Sequential()

    # Input layer
    model.add(Dense(
        units = params['units_1'],
        activation = params['activation'],
        kernel_regularizer = tf.keras.regularizers.l1_l2(l1 = params['l1'], l2 = params['l2']),
        input_dim = X_train.shape[1]
    ))

    # Add dropout after first layer
    model.add(Dropout(params['dropout_1']))

    # Hidden layer
    for i in range(params['n_layers']):
        model.add(Dense(
            units = params[f'units_{i+2}'],
            activation = params['activation'],
            kernel_regularizer = tf.keras.regularizers.l1_l2(l1 = params['l1'], l2 = params['l2'])
        ))
        model.add(Dropout(params[f'dropout_{i+1}']))
    # Output layer (binary classification)
    model.add(Dense(1, activation = 'sigmoid'))

    #compile the model
    optimizer_name = params['optimizer']
    learning_rate = params['learning_rate']

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate = learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate = learning_rate, momentum = params['momentum'])
    else:
        optimizer = RMSprop(learning_rate = learning_rate)

    model.compile(
        optimizer = optimizer,
        loss = 'binary_crossentropy',
        metrics = ['accuracy', tf.keras.metrics.AUC(name = 'auc')]
    )
    return model
        